In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
from matplotlib.font_manager import FontProperties
from matplotlib import rc
import matplotlib as mpl
import sys
sys.path.append("..")
from exotop import topo_1D_plots as plottop
from exotop.model_1D import inputs as ins
from exotop.model_1D import parameters as p

# set paths
# fig_path = '/space/cmg76/figs/' # office
fig_path = 'figs/' # laptop#
fig_format = '.png'
benchmark_path = '../benchmarks/'

# set mpl display parameters
mpl.rcParams['axes.linewidth'] = 2  # set the value globally - for poster
rc('font',**{'family':'serif','serif':['Times New Roman']})  # latex font for matplotlib CMU Serif Roman
# rc('font',**{'family':'monospace','monospace':['Ubuntu Mono']})  # for computery poster 
rc('text', usetex=True)  # turn on for latexy font
mpl.font_manager._rebuild()
plt.style.use('dark_background')

In [1]:
import pandas as pd
import numpy as np

filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/diamonds.csv'
df = pd.read_csv(filepath).iloc[:, [0,4,6]]
df.head()

,carat,depth,price
0,0.23,61.5,326
1,0.21,59.8,326
2,0.23,56.9,327
3,0.29,62.4,334
4,0.31,63.3,335


In [8]:
def mahalanobis(x=None, data=None, cov=None):
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    x_minus_mu = x - np.mean(data)
    print('mean data\n', np.mean(data))
    print('x minus mu\n', x_minus_mu)
    if not cov:
        cov = np.cov(data.values.T)
    print('cov\n', cov)
    inv_covmat = np.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)
    mahal = np.dot(left_term, x_minus_mu.T)
    return mahal.diagonal()

df_x = df[['carat', 'depth', 'price']].head(500)
# print('df_x\n', df_x)
# print('df\n',df[['carat', 'depth', 'price']])
df_x['mahala'] = mahalanobis(x=df_x, data=df[['carat', 'depth', 'price']])
df_x.head()

mean data
 carat       0.797940
depth      61.749405
price    3932.799722
dtype: float64
x minus mu
        carat     depth        price
0   -0.56794 -0.249405 -3606.799722
1   -0.58794 -1.949405 -3606.799722
2   -0.56794 -4.849405 -3605.799722
3   -0.50794  0.650595 -3598.799722
4   -0.48794  1.550595 -3597.799722
..       ...       ...          ...
495 -0.08794  0.950595 -1110.799722
496 -0.08794 -0.449405 -1110.799722
497 -0.09794 -1.549405 -1110.799722
498 -0.09794 -1.049405 -1110.799722
499  0.10206  2.250595 -1110.799722

[500 rows x 3 columns]
cov
 [[ 2.24686660e-01  1.91665282e-02  1.74276536e+03]
 [ 1.91665282e-02  2.05240384e+00 -6.08537121e+01]
 [ 1.74276536e+03 -6.08537121e+01  1.59156294e+07]]


,carat,depth,price,mahala
0,0.23,61.5,326,1.709860
1,0.21,59.8,326,3.540097
2,0.23,56.9,327,12.715021
3,0.29,62.4,334,1.454469
4,0.31,63.3,335,2.347239
